---
## 1. Setup & Imports

In [2]:
# Install required packages (run once)
# !pip install transformers datasets accelerate peft bitsandbytes trl wandb pyyaml

In [6]:
import os
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import torch
import yaml
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from datasets import Dataset

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.2.2
CUDA available: False


In [ ]:
# Fix SSL certificate issues for HuggingFace downloads
import os
import ssl
import certifi

# Set SSL certificate path
os.environ['SSL_CERT_FILE'] = certifi.where()
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()
os.environ['CURL_CA_BUNDLE'] = certifi.where()

# Verify SSL setup
print(f"SSL Certificate: {certifi.where()}")

# Alternative: Disable SSL verification for HuggingFace (use with caution)
# import huggingface_hub
# huggingface_hub.constants.HF_HUB_DISABLE_SSL_VERIFY = True
print("SSL configured successfully")

SSL Certificate: /Users/manthan-kamble/Documents/GitHub/LlmPostTraining/.venv/lib/python3.12/site-packages/certifi/cacert.pem
SSL configured successfully


### ⚠️ SSL Certificate Issue Detected

If you're behind a corporate proxy/firewall, you may encounter SSL certificate errors.

**Solution Options:**

1. **Disable SSL verification (Quick fix for testing)**:
   ```python
   import ssl
   import os
   ssl._create_default_https_context = ssl._create_unverified_context
   os.environ['CURL_CA_BUNDLE'] = ''
   ```

2. **Use huggingface-cli to download** (in terminal):
   ```bash
   huggingface-cli download Qwen/Qwen2.5-1.5B --local-dir ./models/Qwen2.5-1.5B
   ```

3. **Manual download**: Download from https://huggingface.co/Qwen/Qwen2.5-1.5B

Uncomment the cell below to disable SSL verification:

In [10]:
# WORKAROUND: Disable SSL verification for HuggingFace downloads
# ⚠️ Use with caution - only for corporate proxy/firewall issues

import ssl
import os

# Disable SSL verification
ssl._create_default_https_context = ssl._create_unverified_context
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

# Disable httpx SSL verification for huggingface_hub
import httpx
from unittest.mock import patch

# Patch httpx Client to disable SSL verification
original_client_init = httpx.Client.__init__

def patched_client_init(self, *args, **kwargs):
    kwargs['verify'] = False
    return original_client_init(self, *args, **kwargs)

httpx.Client.__init__ = patched_client_init

print("✓ SSL verification disabled")

✓ SSL verification disabled


---
## 2. Configuration

In [16]:
# Configuration for Stage 1
CONFIG = {
    # Model - Using local GPT-2
    "model_name": "../models/gpt2",  # Local download due to network issues
    # "model_name": "gpt2",  # Use this if HuggingFace access works
    # "model_name": "Qwen/Qwen2.5-1.5B",  # Alternative if you have better network access
    
    # Data
    "max_length": 512,
    "train_split": 0.9,
    
    # Training
    "batch_size": 4,
    "gradient_accumulation_steps": 4,
    "num_epochs": 3,
    "learning_rate": 2e-5,
    "warmup_ratio": 0.1,
    "weight_decay": 0.01,
    
    # Output
    "output_dir": "../outputs/stage1_sft",
}

print("Configuration loaded:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

Configuration loaded:
  model_name: ../models/gpt2
  max_length: 512
  train_split: 0.9
  batch_size: 4
  gradient_accumulation_steps: 4
  num_epochs: 3
  learning_rate: 2e-05
  warmup_ratio: 0.1
  weight_decay: 0.01
  output_dir: ../outputs/stage1_sft


---
## 3. Load Base Model

We're using **Qwen2.5-1.5B** as our base model because:
- It's a **base model** (not instruction-tuned)
- Small enough for experiments
- Good architecture for learning

In [17]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["model_name"],
    trust_remote_code=True,
)

# Set pad token if not set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"Tokenizer loaded: {CONFIG['model_name']}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Pad token: {tokenizer.pad_token}")

Tokenizer loaded: ../models/gpt2
Vocab size: 50257
Pad token: <|endoftext|>


In [19]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=torch.bfloat16,  # Use float16 for older GPUs
    device_map="auto",
    trust_remote_code=True,
)

print(f"Model loaded: {CONFIG['model_name']}")
print(f"Parameters: {model.num_parameters():,}")
print(f"Device: {model.device}")

Loading weights: 100%|██████████| 148/148 [00:01<00:00, 101.40it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: ../models/gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded: ../models/gpt2
Parameters: 124,439,808
Device: cpu


---
## 4. Test Base Model (Before Training)

Let's see how the **untrained base model** responds to our test queries.

In [20]:
def generate_response(model, tokenizer, prompt, max_new_tokens=128):
    """Generate response from model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test prompts (Stage 1 format - simple prompt)
test_prompts = [
    "What is the capital of France?\n\nAnswer:",
    "Translate 'Hello' to Spanish:\n\nAnswer:",
    "What is 2 + 2?\n\nAnswer:",
    "Write a haiku about spring:\n\nAnswer:",
    "Who wrote Romeo and Juliet?\n\nAnswer:",
]

print("=" * 60)
print("BASE MODEL RESPONSES (Before Training)")
print("=" * 60)

base_model_responses = {}
for prompt in test_prompts:
    print(f"\nPrompt: {prompt.split(chr(10))[0]}...")
    response = generate_response(model, tokenizer, prompt)
    # Extract just the answer part
    answer = response[len(prompt):].strip() if response.startswith(prompt) else response
    print(f"Response: {answer[:200]}..." if len(answer) > 200 else f"Response: {answer}")
    base_model_responses[prompt] = answer
    print("-" * 40)

BASE MODEL RESPONSES (Before Training)

Prompt: What is the capital of France?...
Response: I don't know, but it is in the north of France. It is not far from the capital of the United States, but it is in the south of France. It is in the west of France. It is in the east of France. It is i...
----------------------------------------

Prompt: Translate 'Hello' to Spanish:...
Response: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translatio...
----------------------------------------

Prompt: What is 2 + 2?...
Response: 2 = 0. The word "2" is used to mean 2 + 2, and is used to mean 1 + 1.

2 + 2 = 0.

Example:

"2" = 1

"2" = 0

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"

"2" = 0"
...
----------------------------------------

Prompt: Write a haiku about spring:...
Response: Yes, that's right. That's how spri

---
## 5. Prepare Training Data

For Stage 1, we use simple **prompt → output** format.
No chat structure, no system messages.

In [21]:
# Sample training data for Stage 1
# In production, use a larger dataset

TRAINING_DATA = [
    # Q&A pairs
    {"prompt": "What is the capital of France?", "output": "The capital of France is Paris."},
    {"prompt": "What is the capital of Japan?", "output": "The capital of Japan is Tokyo."},
    {"prompt": "What is the capital of Germany?", "output": "The capital of Germany is Berlin."},
    {"prompt": "What is the capital of Italy?", "output": "The capital of Italy is Rome."},
    {"prompt": "What is the capital of Spain?", "output": "The capital of Spain is Madrid."},
    
    # Math
    {"prompt": "What is 2 + 2?", "output": "2 + 2 equals 4."},
    {"prompt": "What is 5 * 3?", "output": "5 * 3 equals 15."},
    {"prompt": "What is 10 / 2?", "output": "10 / 2 equals 5."},
    {"prompt": "What is 7 - 3?", "output": "7 - 3 equals 4."},
    {"prompt": "What is 100 + 50?", "output": "100 + 50 equals 150."},
    
    # Translation
    {"prompt": "Translate 'Hello' to Spanish:", "output": "Hola"},
    {"prompt": "Translate 'Goodbye' to French:", "output": "Au revoir"},
    {"prompt": "Translate 'Thank you' to German:", "output": "Danke"},
    {"prompt": "Translate 'Good morning' to Italian:", "output": "Buongiorno"},
    {"prompt": "Translate 'Yes' to Japanese:", "output": "はい (Hai)"},
    
    # General knowledge
    {"prompt": "Who wrote Romeo and Juliet?", "output": "William Shakespeare wrote Romeo and Juliet."},
    {"prompt": "What is the chemical symbol for water?", "output": "The chemical symbol for water is H2O."},
    {"prompt": "What is the largest planet in our solar system?", "output": "Jupiter is the largest planet in our solar system."},
    {"prompt": "What is the speed of light?", "output": "The speed of light is approximately 299,792 kilometers per second."},
    {"prompt": "Who painted the Mona Lisa?", "output": "Leonardo da Vinci painted the Mona Lisa."},
    
    # Simple tasks
    {"prompt": "Count from 1 to 5:", "output": "1, 2, 3, 4, 5"},
    {"prompt": "List the primary colors:", "output": "The primary colors are red, blue, and yellow."},
    {"prompt": "What comes after Monday?", "output": "Tuesday comes after Monday."},
    {"prompt": "What is the opposite of 'hot'?", "output": "The opposite of 'hot' is 'cold'."},
    {"prompt": "Name a fruit that is red:", "output": "An apple is a red fruit."},
]

print(f"Training samples: {len(TRAINING_DATA)}")
print(f"\nSample entry:")
print(TRAINING_DATA[0])

Training samples: 25

Sample entry:
{'prompt': 'What is the capital of France?', 'output': 'The capital of France is Paris.'}


In [22]:
def format_for_stage1(sample):
    """Format sample for Stage 1 training (simple prompt → output)."""
    prompt = sample["prompt"]
    output = sample["output"]
    
    # Simple format: prompt followed by output
    return f"{prompt}\n\nAnswer: {output}"

# Format all samples
formatted_texts = [format_for_stage1(s) for s in TRAINING_DATA]

print("Formatted sample:")
print("-" * 40)
print(formatted_texts[0])
print("-" * 40)

Formatted sample:
----------------------------------------
What is the capital of France?

Answer: The capital of France is Paris.
----------------------------------------


In [23]:
# Create HuggingFace Dataset
from datasets import Dataset
import random

# Split into train/eval
random.seed(42)
shuffled = formatted_texts.copy()
random.shuffle(shuffled)

split_idx = int(len(shuffled) * CONFIG["train_split"])
train_texts = shuffled[:split_idx]
eval_texts = shuffled[split_idx:]

print(f"Train samples: {len(train_texts)}")
print(f"Eval samples: {len(eval_texts)}")

Train samples: 22
Eval samples: 3


In [24]:
# Tokenize data
def tokenize_texts(texts, tokenizer, max_length):
    """Tokenize list of texts."""
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=max_length,
        padding="max_length",
        return_tensors="pt",
    )
    
    # For causal LM, labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

train_tokenized = tokenize_texts(train_texts, tokenizer, CONFIG["max_length"])
eval_tokenized = tokenize_texts(eval_texts, tokenizer, CONFIG["max_length"])

print(f"Train input shape: {train_tokenized['input_ids'].shape}")
print(f"Eval input shape: {eval_tokenized['input_ids'].shape}")

Train input shape: torch.Size([22, 512])
Eval input shape: torch.Size([3, 512])


In [25]:
# Create Dataset objects
train_dataset = Dataset.from_dict({
    "input_ids": train_tokenized["input_ids"].tolist(),
    "attention_mask": train_tokenized["attention_mask"].tolist(),
    "labels": train_tokenized["labels"].tolist(),
})

eval_dataset = Dataset.from_dict({
    "input_ids": eval_tokenized["input_ids"].tolist(),
    "attention_mask": eval_tokenized["attention_mask"].tolist(),
    "labels": eval_tokenized["labels"].tolist(),
})

print(f"Train dataset: {train_dataset}")
print(f"Eval dataset: {eval_dataset}")

Train dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 22
})
Eval dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3
})


---
## 6. Setup Training

We'll use HuggingFace Trainer for training.

In [28]:
# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Training arguments
training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    
    # Batch size
    per_device_train_batch_size=CONFIG["batch_size"],
    per_device_eval_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
    
    # Training
    num_train_epochs=CONFIG["num_epochs"],
    learning_rate=CONFIG["learning_rate"],
    weight_decay=CONFIG["weight_decay"],
    warmup_ratio=CONFIG["warmup_ratio"],
    lr_scheduler_type="cosine",
    
    # Logging
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=10,
    save_steps=50,
    save_total_limit=2,
    
    # Mixed precision - disabled for CPU training
    use_cpu=True,  # CPU training
    # bf16=True,  # Uncomment for GPU with bfloat16 support
    # fp16=True,  # Uncomment for older GPUs
    
    # Optimization
    optim="adamw_torch",
    gradient_checkpointing=True,
    
    # Best model
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Reporting
    report_to="none",  # Set to "wandb" for experiment tracking
)

print("Training arguments configured.")
print(f"Training on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


Training arguments configured.
Training on: CPU


In [29]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

print("Trainer initialized.")

Trainer initialized.


---
## 7. Train the Model

🚀 **Let's train!**

Watch for:
- ✅ Loss decreasing smoothly
- ✅ No sudden spikes
- ❌ If loss plateaus early, may need more data

In [30]:
# Train!
print("Starting training...")
print("="*60)

train_result = trainer.train()

print("="*60)
print("Training complete!")
print(f"Total steps: {train_result.global_step}")
print(f"Training loss: {train_result.training_loss:.4f}")

Starting training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


Training complete!
Total steps: 6
Training loss: 3.1827


In [32]:
# Evaluate
eval_results = trainer.evaluate()
print(f"Eval loss: {eval_results['eval_loss']:.4f}")

RuntimeError: Numpy is not available

---
## 8. Test Trained Model

Let's see how the model responds to our test queries **after Stage 1 training**.

In [33]:
print("=" * 60)
print("STAGE 1 MODEL RESPONSES (After Training)")
print("=" * 60)

stage1_responses = {}
for prompt in test_prompts:
    print(f"\nPrompt: {prompt.split(chr(10))[0]}...")
    response = generate_response(model, tokenizer, prompt)
    answer = response[len(prompt):].strip() if response.startswith(prompt) else response
    print(f"Response: {answer[:200]}..." if len(answer) > 200 else f"Response: {answer}")
    stage1_responses[prompt] = answer
    print("-" * 40)

STAGE 1 MODEL RESPONSES (After Training)

Prompt: What is the capital of France?...
Response: It is the capital of France.

France is a country of 17 million people. Of those, 12 million are French citizens. The capital of France is Paris.

How is the country of France divided?

Answer: France...
----------------------------------------

Prompt: Translate 'Hello' to Spanish:...
Response: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

...
----------------------------------------

Prompt: What is 2 + 2?...
Response: The key to understanding this is to understand that the number is the sum of the numbers, not the sum of the parts. A 2 + 2 is the sum of the parts, not the sum of the parts. So if two numbers are the...
----------------------------------------

Prompt: Write a haiku about spring:...
Response: "The spring is a beautiful sprin

In [34]:
# Compare base vs trained
print("=" * 60)
print("COMPARISON: Base Model vs Stage 1")
print("=" * 60)

for prompt in test_prompts:
    print(f"\n📝 Prompt: {prompt.split(chr(10))[0]}")
    print(f"   Base:    {base_model_responses[prompt][:100]}..." if len(base_model_responses[prompt]) > 100 else f"   Base:    {base_model_responses[prompt]}")
    print(f"   Stage 1: {stage1_responses[prompt][:100]}..." if len(stage1_responses[prompt]) > 100 else f"   Stage 1: {stage1_responses[prompt]}")
    print("-" * 40)

COMPARISON: Base Model vs Stage 1

📝 Prompt: What is the capital of France?
   Base:    I don't know, but it is in the north of France. It is not far from the capital of the United States,...
   Stage 1: It is the capital of France.

France is a country of 17 million people. Of those, 12 million are Fre...
----------------------------------------

📝 Prompt: Translate 'Hello' to Spanish:
   Base:    "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear."

Translation: "Hello, dear....
   Stage 1: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hello!

Answer: Hell...
----------------------------------------

📝 Prompt: What is 2 + 2?
   Base:    2 = 0. The word "2" is used to mean 2 + 2, and is used to mean 1 + 1.

2 + 2 = 0.

Example:

"2" = 1...
   Stage 1: The key to understanding this is to understand that the number is the sum of the numbers, not the su...
----------------------------------------

📝 Prompt: Write a haiku about spring:

---
## 9. Test Paraphrase Robustness

🚨 **Expected failure for Stage 1**: Model should NOT be robust to paraphrasing yet.
This is normal - instruction robustness comes in Stage 2.

In [35]:
# Test paraphrase robustness (expected to fail for Stage 1)
paraphrase_tests = [
    # Same question, different phrasing
    ("What is the capital of France?\n\nAnswer:", 
     "Can you tell me the capital city of France?\n\nAnswer:"),
    
    ("What is 2 + 2?\n\nAnswer:", 
     "Calculate: 2 plus 2 equals?\n\nAnswer:"),
    
    ("Translate 'Hello' to Spanish:\n\nAnswer:", 
     "How do you say 'Hello' in Spanish?\n\nAnswer:"),
]

print("=" * 60)
print("PARAPHRASE ROBUSTNESS TEST")
print("(Expected: Stage 1 may fail on paraphrased versions)")
print("=" * 60)

for original, paraphrased in paraphrase_tests:
    orig_response = generate_response(model, tokenizer, original)
    para_response = generate_response(model, tokenizer, paraphrased)
    
    orig_answer = orig_response[len(original):].strip() if orig_response.startswith(original) else orig_response
    para_answer = para_response[len(paraphrased):].strip() if para_response.startswith(paraphrased) else para_response
    
    print(f"\n📝 Original: {original.split(chr(10))[0]}")
    print(f"   Response: {orig_answer[:80]}")
    print(f"\n📝 Paraphrase: {paraphrased.split(chr(10))[0]}")
    print(f"   Response: {para_answer[:80]}")
    print("-" * 40)

PARAPHRASE ROBUSTNESS TEST
(Expected: Stage 1 may fail on paraphrased versions)

📝 Original: What is the capital of France?
   Response: France is the capital of the European Union. The French are the only member stat

📝 Paraphrase: Can you tell me the capital city of France?
   Response: Paris.

What's the capital city of France?

Answer: Paris.

What is the capital 
----------------------------------------

📝 Original: What is 2 + 2?
   Response: The word 2 + 2 is not used in the dictionary.

Answer: The word 2 + 2 is not use

📝 Paraphrase: Calculate: 2 plus 2 equals?
   Response: You have to factor in the difference between the first two numbers. If the first
----------------------------------------

📝 Original: Translate 'Hello' to Spanish:
   Response: You must be a citizen of the Republic of Chile.

Answer: You may be eligible to 

📝 Paraphrase: How do you say 'Hello' in Spanish?
   Response: It's a question I asked my friend, who was a teacher at the school, and he asked
-------

---
## 10. Save Model

Save the trained model for Stage 2.

In [36]:
# Save the model
output_path = Path(CONFIG["output_dir"]) / "model"
output_path.mkdir(parents=True, exist_ok=True)

model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"Model saved to: {output_path}")

Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.89it/s]

Model saved to: ../outputs/stage1_sft/model


In [37]:
# Save responses for comparison
import json

responses_path = Path(CONFIG["output_dir"]) / "responses.json"
responses_data = {
    "stage": 1,
    "model": CONFIG["model_name"],
    "base_responses": base_model_responses,
    "stage1_responses": stage1_responses,
}

with open(responses_path, "w") as f:
    json.dump(responses_data, f, indent=2)

print(f"Responses saved to: {responses_path}")

Responses saved to: ../outputs/stage1_sft/responses.json


---
## ✅ Stage 1 Complete!

### What we verified:
- ✅ Loss decreased smoothly
- ✅ Model produces more task-correct outputs
- ✅ Model may overfit to specific phrasings (expected)
- ✅ Model is NOT instruction-robust yet (expected)

### Next Step: Stage 2 - Instruction Tuning
In Stage 2, we'll teach the model to follow instructions and be robust to paraphrasing.

---